# Instalar Librerias

In [ ]:
import numpy as np
from IPython.display import display, Latex
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
#from scipy import signal
#from scipy import fftpack
from importlib import reload
#from scipy import stats

#PARA LAS SEÑALES DE MATLAB
import scipy.io as sio

import csv

import pandas as pd

from sklearn.model_selection import train_test_split

import re

# Montar drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Ejercicio 1:
Consideremos el siguiente vector de atributos binarios: 
 (scones, cerveza, whisky, avena, futbol)
En el archivo PreferenciasBritanicos.xls se encuentran las preferencias de 6 personas inglesas y 7 personas escocesas.

##Codigo

###Obtencion de datos


In [ ]:
path = "/content/drive/Shareddrives/Machine Learning/TP1/PreferenciasBritanicos(1).xlsx"
data1 = pd.read_excel(path)

data1

,scones,cerveza,wiskey,avena,futbol,Nacionalidad
0,0,0,1,1,1,I
1,1,0,1,1,0,I
2,1,1,0,0,1,I
3,1,1,0,0,0,I
4,0,1,0,0,1,I
5,0,0,0,1,0,I
6,1,0,0,1,1,E
7,1,1,0,0,1,E
8,1,1,1,1,0,E
9,1,1,0,1,0,E


In [ ]:
def probabilidades_data(data):
  V_E = []#P(scones.....|E)
  V_I = []
  head = data.columns
  tot_E = data.loc[data['Nacionalidad'] == 'E',head[len(head)-1]].count()
  tot_I = data.loc[data['Nacionalidad'] == 'I',head[len(head)-1]].count()
  
  for j in range(0,len(head)-1):#recorro filas
    Q_ocur_E = data.loc[data['Nacionalidad'] == 'E', head[j]].sum()
    Q_ocur_I = data.loc[data['Nacionalidad'] == 'I', head[j]].sum() 

    aux_E = (Q_ocur_E+1)/(tot_E+2) #aplicamos laplace
    V_E.append(aux_E)
    aux_I = (Q_ocur_I+1)/(tot_I+2) #aplicamos laplace
    V_I.append(aux_I)

  I = data.loc[data['Nacionalidad'] == 'I','Nacionalidad'].count()/len(data)

  return [V_E,V_I,I]

###Funciones clasificador ingenuo de Bayes

In [ ]:
def probabilidad_P_E_I(P,V_E,V_I,I):
  P_E = 1
  P_I = 1
  for i in range(0,len(P)):
    if P[i]:
      P_E *= V_E[i]
      P_I *= V_I[i]
    else:
      P_E *= (1 - V_E[i])
      P_I *= (1 - V_I[i])
  
  return [P_E, P_I] #[P(P|E),P(P|I)]

In [ ]:
def probabilidad_I_E(I,V_E,V_I,P):

  if len(V_E) != len(V_I) or len(V_E) != len(P) or len(P) != len(V_I): 
    E_P = 0
    I_P = 0
    print("El largo de los vectores V_E, V_G y NO ingresados debe ser el mismo.")
  else:
    [P_E, P_I] = probabilidad_P_E_I(P,V_E,V_I,I)

    E_P_ = (P_E * (1-I)) # P(P|E)P(E)
    G_P_ = (P_I * I) # P(P|I)P(I)

    if E_P_ > G_P_: #tomo el valor mayor entre ambos
      rta = 'E'
    else:
      rta = 'I'

  return rta

##Resultados

In [ ]:
data1

,scones,cerveza,wiskey,avena,futbol,Nacionalidad
0,0,0,1,1,1,I
1,1,0,1,1,0,I
2,1,1,0,0,1,I
3,1,1,0,0,0,I
4,0,1,0,0,1,I
5,0,0,0,1,0,I
6,1,0,0,1,1,E
7,1,1,0,0,1,E
8,1,1,1,1,0,E
9,1,1,0,1,0,E


Cada una de las filas de la tabla representa una persona con sus gustos de scones, cerveza, whisky, avena y futbol (siendo 0 que no le gusta y 1 que si) y ademas posee una clasificacion segun su nacionalidad, siendo 'E' Escocesa e 'I' Inglesa.

A partir de la tabla dada vamos a obtener las siguientes probabilidades:
*  $P(V_{j}|E)$ y $P(V_{j}|I)$  donde $V = \{scones, cerveza, whisky, avena, futbol\}$
*  $P(I)$

A partir de:

$p = \frac{cantidad \, de \, ocurrencias}{total}$

A la cual vamos a aplicar la corrección de Laplace para eliminar las probabilidades nulas:

$p = \frac{cantidad \, de \, ocurrencias + 1}{total + k}$

Siendo k la cantidad de clases, para este caso 5.

Inicialmente, en el inciso a, separaremos el total de datos en conjunto de testeo y entrenamiento para calcular distintas metricas del metodo. Las mismas pueden utilizarse para comparar un método de clasificación aplicado a varios conjuntos de datos o para medir el efecto que produce el cambio de parámetros en un método. En este caso las probabilidades a usar van a ser obtenidas por medio del conjunto de entrenamiento. Para el inciso b y c usaremos como conjunto de entrenamiento todos los datos, es decir obtendremos las probabilidades de todas las personas dadas. 

Buscamos clasificar distintas personas segun sus preferencias por los scones, cerveza, whisky, avena y futbol en Escoceses o Ingleses. Para esto vamos a buscar:

* $h_{MAP} = arg max\{ P(P|E) \cdot P(E),  P(P|I) \cdot P(I)\}$

Donde P es la persona y sus preferencias, 'E' Escoces e 'I' Ingles.

Esto proviene de $P(E|P)$ y $P(I|P)$ que por Teorema de Bayes son:

* $P(E|P) = \frac{P(P|E) \cdot P(E)}{P(P)}$ 

* $P(I|P) = \frac{P(P|I) \cdot P(I)}{P(P)}$  

Pero debido a que $P(P)$ es la misma para ambas probabilidades unicamente vamos a buscar el maximo valor del denominador para determinar la nacionalidad de la persona dada.

Para esto buscamos las probabilidades $P(P|E)$ y $P(P|I)$ que su expresion va a depender de el vector de atributos que describe a la persona P, debido a que atributos (scones, cerveza, whisky, avena y futbol) son independientes dadas las nacionalidades de las personas.

## a) Implementar el clasificador ingenuo de Bayes.




###Codigo

In [ ]:
#creo conjunto de entrenamiento y testeo
size_test1= 0.3
test1,train1 = train_test_split(data1, train_size = size_test1, random_state = 1) #data es mi conjunto de entrenamiento

In [ ]:
#obtengo datos del conjunto entrenamiento
[train_V_E,train_V_I,train_I] = probabilidades_data(train1) 

train1

,scones,cerveza,wiskey,avena,futbol,Nacionalidad
2,1,1,0,0,1,I
3,1,1,0,0,0,I
4,0,1,0,0,1,I
10,1,1,0,1,1,E
1,1,0,1,1,0,I
6,1,0,0,1,1,E
0,0,0,1,1,1,I
7,1,1,0,0,1,E
12,1,0,1,0,0,E
9,1,1,0,1,0,E


In [ ]:
test1

,scones,cerveza,wiskey,avena,futbol,Nacionalidad
8,1,1,1,1,0,E
11,1,0,1,1,0,E
5,0,0,0,1,0,I


In [ ]:
#obtengo las nacionalidades de las personas en el grupo de testeo mediante el metodo ingenuo de Bayes 
#y las comparo con las reales
nac_real = []
nac_pred = []
indep = False
for i in test1.index:
  nac_real.append(test1.loc[i,'Nacionalidad'])
  nac_pred.append(probabilidad_I_E(train_I,train_V_E,train_V_I,test1.loc[i,'scones':'futbol']))

tabla_nac= pd.DataFrame()
tabla_nac['Nacionalidad real'] = nac_real
tabla_nac['Nacionalidad predicha'] = nac_pred

tabla_nac

,Nacionalidad real,Nacionalidad predicha
0,E,E
1,E,E
2,I,I


In [ ]:
#creamos matriz de confusion predichos en columnas y reales en filas
head1 = ['Escoces','Ingles']
matriz_confusion1 = pd.DataFrame(columns = head1,index = head1)
matriz_confusion1 = matriz_confusion1.fillna(0)
tabla_nac_dis = tabla_nac.copy()
tabla_nac_dis.loc[tabla_nac_dis['Nacionalidad real']=='E'] = 0
tabla_nac_dis.loc[tabla_nac_dis['Nacionalidad real']=='I'] = 1

for i in range(0,len(tabla_nac)):
  matriz_confusion1.iloc[tabla_nac_dis.iloc[i,0],tabla_nac_dis.iloc[i,1]] += 1

matriz_confusion1

,Escoces,Ingles
Escoces,2,0
Ingles,0,1


In [ ]:
head1_ = head1.copy()
head1_.append('Total')
head1_

['Escoces', 'Ingles', 'Total']

In [ ]:
#contamos VP,VN,FP,FN
tabla_VF1 = pd.DataFrame(columns = ['VP','VN','FP','FN'], index = head1_)
tabla_VF1 = tabla_VF1.fillna(0)

for i in range(0,2):#filas
  for j in range(0,2):#columnas
    if i == j:
      tabla_VF1.iloc[i,0] = matriz_confusion1.iloc[i,j]
    else:
      tabla_VF1.iloc[i,3] += matriz_confusion1.iloc[i,j]
      tabla_VF1.iloc[j,2] += matriz_confusion1.iloc[i,j]

for i in range(0,2):
  tabla_VF1.iloc[i,1] = matriz_confusion1.sum().sum()-tabla_VF1.sum(axis=1)[i]

for i in range(0,4):
  tabla_VF1.iloc[2,i] = tabla_VF1.sum()[i]
tabla_VF1

,VP,VN,FP,FN
Escoces,2,1,0,0
Ingles,1,2,0,0
Total,3,3,0,0


In [ ]:
#metricas
head_metricas1 = ['Accuracy','Precision','Recall','F1-Score','Tasa de FP']
metricas1 = pd.DataFrame(columns= head_metricas1, index=head1_)
metricas1 = metricas1.fillna(0)

metricas1

for i in metricas1.index:
  #Accuracy
  metricas1.loc[i,'Accuracy'] = (tabla_VF1.loc[i,'VP'] + tabla_VF1.loc[i,'VN'])/(tabla_VF1.loc[i,'VP']+tabla_VF1.loc[i,'VN']+tabla_VF1.loc[i,'FP']+tabla_VF1.loc[i,'FN'])
  #Precision
  metricas1.loc[i,'Precision'] = tabla_VF1.loc[i,'VP']/(tabla_VF1.loc[i,'VP']+tabla_VF1.loc[i,'FP'])
  #Recall
  metricas1.loc[i,'Recall'] = tabla_VF1.loc[i,'VP']/(tabla_VF1.loc[i,'VP']+tabla_VF1.loc[i,'FN'])
  #F1-Score
  metricas1.loc[i,'F1-Score'] = (2*metricas1.loc[i,'Precision']*metricas1.loc[i,'Recall'])/(metricas1.loc[i,'Precision']+metricas1.loc[i,'Recall'])
  #Tasa de FP
  metricas1.loc[i,'Tasa de FP'] = tabla_VF1.loc[i,'FP']/(tabla_VF1.loc[i,'FP']+tabla_VF1.loc[i,'VN'])

metricas1

,Accuracy,Precision,Recall,F1-Score,Tasa de FP
Escoces,1.0,1.0,1.0,1.0,0.0
Ingles,1.0,1.0,1.0,1.0,0.0
Total,1.0,1.0,1.0,1.0,0.0


###Resultados

Separamos los datos en conjunto de entrenamiento (70%) y testeo (30%). Siendo los conjuntos obtenidos los siguientes:

In [ ]:
train1

,scones,cerveza,wiskey,avena,futbol,Nacionalidad
2,1,1,0,0,1,I
3,1,1,0,0,0,I
4,0,1,0,0,1,I
10,1,1,0,1,1,E
1,1,0,1,1,0,I
6,1,0,0,1,1,E
0,0,0,1,1,1,I
7,1,1,0,0,1,E
12,1,0,1,0,0,E
9,1,1,0,1,0,E


In [ ]:
test1

,scones,cerveza,wiskey,avena,futbol,Nacionalidad
8,1,1,1,1,0,E
11,1,0,1,1,0,E
5,0,0,0,1,0,I


Luego calculamos por el metodo ingenuo de Bayes y las probabilidades obtenidas con el conjunto de entrenamiento las Nacionalidades de las personas endel conjunto de testeo y armamos una tabla con las mismas y las nacionalidades reales:

In [ ]:
tabla_nac

,Nacionalidad real,Nacionalidad predicha
0,E,E
1,E,E
2,I,I


Con dicha informacion de 'tabla_nac' podemos realizar la matriz de confusion, donde las columnas son las nacionalidades predichas y las filas las nacionalidades reales.

In [ ]:
matriz_confusion1

,Escoces,Ingles
Escoces,2,0
Ingles,0,1


Luego a partir de la matriz de confusion obtenemos los valores 'VP' 'VN' 'FP' 'FN', obteniendo:

In [ ]:
tabla_VF1

,VP,VN,FP,FN
Escoces,2,1,0,0
Ingles,1,2,0,0
Total,3,3,0,0


Finalmente a partir de la tabla 'tabla_VF1' calculamos distintas metricas, las cuales son:

* $Accuracy  = \frac{VP + VN}{VP+VN+FP+FN}$

* $Precision  = \frac{VP}{VP+FP}$

* $Recall = Tasa \, de \, VP = \frac{VP}{VP+FN}$

* $F1-Score  = \frac{2\cdot Precisión \cdot Recall}{Precisión + Recall}$

* $Tasa \, de \, FP  = \frac{FP}{FP+VN}$

Obteninendo asi:


In [ ]:
metricas1

,Accuracy,Precision,Recall,F1-Score,Tasa de FP
Escoces,1.0,1.0,1.0,1.0,0.0
Ingles,1.0,1.0,1.0,1.0,0.0
Total,1.0,1.0,1.0,1.0,0.0


##b) Clasificar el ejemplo x1 = (1; 0; 1; 1; 0) determinando si corresponde a las preferencias de una persona inglesa o escosesa.

In [ ]:
[V_E,V_I,I] = probabilidades_data(data1)

x1 = [1, 0, 1, 1, 0]
info = probabilidad_I_E(I,V_E,V_I,x1)

info

'E'

##c) Clasificar el ejemplo x2 = (0; 1; 1; 0; 1), determinando si corresponde a las preferencias de una persona inglesa o escosesa.

In [ ]:
x2 = [0, 1, 1, 0, 1]
info2 = probabilidad_I_E(I,V_E,V_I,x2)
info2

'I'